In [1]:
import requests
import random
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from IPython.display import Image
from IPython.core.display import HTML
from pandas.io.json import json_normalize

import folium

print("imported and donloaded")

imported and donloaded


In [10]:
client_id = 'VOTIKZVKJBNCDMZ3W4HQESWMXTUJM141QV3I35FGOYRID1I1'
client_secret = 'PA5VTALFZBRUMWCTGPLFMKF0GMDI1WQUYR3SUHLDOAU0DT3E'
version = '20201215'
limit = 30

In [11]:
address = '102 North End Ave, New York,NY'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude,longitude)

40.7149555 -74.0153365


In [16]:
search_query = 'Italian'
radius = 500

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(client_id,client_secret,latitude,longitude,version,search_query,radius,limit)
url

'https://api.foursquare.com/v2/venues/search?client_id=VOTIKZVKJBNCDMZ3W4HQESWMXTUJM141QV3I35FGOYRID1I1&client_secret=PA5VTALFZBRUMWCTGPLFMKF0GMDI1WQUYR3SUHLDOAU0DT3E&ll=40.7149555,-74.0153365&v=20201215&query=Italian&radius=500&limit=30'

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd8dcd90223702a3a0dd9d6'},
 'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351},
      {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
     'distance': 58,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-16080478

In [18]:
venues = results['response']['venues']

In [19]:
dataframe = json_normalize(venues)
dataframe.head()

<ipython-input-19-da7c61c13521>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1608047833,False,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN
1,4f3232e219836c91c7bfde94,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1608047833,False,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN
2,3fd66200f964a520f4e41ee3,Ecco,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1608047833,False,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway


In [36]:
dataframe['categories'][1]

[{'id': '4d4b7105d754a06374d81259',
  'name': 'Food',
  'pluralName': 'Food',
  'shortName': 'Food',
  'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
   'suffix': '.png'},
  'primary': True}]

In [20]:
filtered_columns = ['name','categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:,filtered_columns]

In [21]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

In [22]:
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type,axis=1)

In [23]:
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

In [24]:
dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway,3fd66200f964a520f4e41ee3


In [41]:
dataframe_filtered.name
venues_map = folium.Map(location=[latitude,longitude], zoom_start=13)

In [42]:
folium.CircleMarker([latitude,longitude],radius=10, color='red', popup = 'Conrad Hotel', fill = True, fill_color = 'red', fill_opacity = 0.6).add_to(venues_map)

In [44]:
for lat,long,label in zip(dataframe_filtered.lat,dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat,long],
        radius = 5,
        color='blue',
        popup = label,
        fill = True,
        fill_color = 'blue',
        fill_opacity = 0.6
        ).add_to(venues_map)

In [46]:
venues_map